In [ ]:
require 'nn'
require 'hdf5'

# Loading data

In [2]:
-- Train format is (number of Ngrams, Ngram_size + 1) with last
-- col the count of the N_gram of the line

-- Validation format is (number of words to predict, 50 + Ngrams_size -1)
-- where the 50 columns stands for the 50 words possibilities in the prediction,
-- the next col stands for the current context (goal is to predict the Nth word)

myFile = hdf5.open('1-grams.hdf5','r')
data = myFile:all()
train_1 = data['train']
myFile:close()

myFile = hdf5.open('2-grams.hdf5','r')
data = myFile:all()
train_2 = data['train']
validation_2 = data['valid']
validation_output = data['valid_output']
myFile:close()

myFile = hdf5.open('3-grams.hdf5','r')
data = myFile:all()
train_3 = data['train']
validation_3 = data['valid']
myFile:close()

myFile = hdf5.open('4-grams.hdf5','r')
data = myFile:all()
train_4 = data['train']
validation_4 = data['valid']
myFile:close()


# Maximum Likelihood Estimation

In [ ]:
-- Second version: reversing the order in F_c_w

function build_context_count(count_tensor)
    -- Ngram count (depend on w and context)
    -- {'index1-...-indexN-1': {'indexN' : count}}
    local F_c_w = {}
    -- F_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count all words in c}
    local F_c = {}
    -- N_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count unique type of words in c}
    local N_c = {}

    local N = count_tensor:size(1)
    local M = count_tensor:size(2)

    for i=1, N do
        indexN = count_tensor[{i,M-1}]
        
        -- build the key index1-...-indexN-1
        indexes = tostring(count_tensor[{i,1}])
        for j=2, M - 2 do
            indexes = indexes .. '-' .. tostring(count_tensor[{i,j}])
        end
        
        -- Filling F_c_w
        if F_c_w[indexes] == nil then
            F_c_w[indexes] = {[indexN] = count_tensor[{i, M}]}
        else
            F_c_w[indexes][indexN] = count_tensor[{i, M}]
        end
        
        -- Updating F_c and F_c
        if F_c[indexes] == nil then
            F_c[indexes] = count_tensor[{i, M}]
            N_c[indexes] = 1
        else
            F_c[indexes] = count_tensor[{i, M}] + F_c[indexes]
            N_c[indexes] = 1 + N_c[indexes]
        end
    end
    
    return F_c_w, F_c, N_c
end

In [5]:
-- Prediction with the MLE (with Laplace smoothing)

function mle_proba(data, F_c_w, alpha)
    local N = data:size(1)
    local M = data:size(2)
    -- Output format: distribution predicted for each N word along the
    -- 50 possibilities
    local distribution = torch.DoubleTensor(N, 50)

    for i=1, N do
        -- build the key index1-...-indexN-1
        indexes = tostring(data[{i,51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i,j}])
        end
        
        -- Look up in the dictionnary for the 50 possible ngrams asked
        for j=1, 50 do
            indexN = data[{i,j}]
            if F_c_w[indexN] == nil or F_c_w[indexN][indexes] == nil then
                distribution[{i,j}] = alpha
            else
                distribution[{i,j}] = F_c_w[indexN][indexes] + alpha
            end
        end
        -- Debug: case where no n-gram were found (only when alpha=0.)
        if distribution:narrow(1,i,1):sum(2)[{1,1}] == 0 then
            -- Select the first one (most common)
            distribution[{i,1}] = 1
        end
    end
    -- normalization (ie we do the MLE given only the 50 possibilities)
    distribution:cdiv(torch.expand(distribution:sum(2), N, 50))
    
    return distribution
end

In [6]:
-- Loading train of the gram_size N
function get_train(N)
    local filename = N .. '-grams.hdf5'
    print(filename)
    myFile = hdf5.open(filename,'r')
    train = myFile:all()['train']
    myFile:close()
    return train
end

In [7]:
-- Prediction with the MLE (with Laplace smoothing) and fix on never seen context

function mle_proba_2(data, N, alpha)
    local N_data = data:size(1)
    local M = data:size(2)
    -- Output format: distribution predicted for each N_data word along the
    -- 50 possibilities
    local distribution = torch.DoubleTensor(N_data, 50)
    local gram_size
    
    -- Building the count matrix for each ngram size lower than N,
    -- if need to reduce the context in case of unseen new context
    local F_c_w_table = {}
    for i=1,N do
        train = get_train(i)
        F_c_w, F_c, N_c = build_context_count_2(train)
        F_c_w_table[i] = F_c_w
    end

    for i=1, N_data do
        -- Initialize the Ngram_size
        gram_size = N
        -- build the key index1-...-indexN-1
        indexes = tostring(data[{i,51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i,j}])
        end
        -- case context never seen before:
        -- we look for the reduced context (ie index2-...-indexN-1)
        if F_c_w_table[N][indexes] == nil then
            -- Look for previous context
            for k = N-1,1,-1 do
                -- update gram_size
                gram_size = k
                -- Building index in reduced context
                indexes_split = indexes:split("-")
                indexes = ''
                for i= 1 + N - gram_size, N-1 do
                    indexes = indexes .. indexes_split[i]
                end
                -- look if current context seen
                if F_c_w_table[gram_size][indexes] ~= nil then
                    break
                end
            end 
        end

        -- Look up in the dictionnary for the 50 possible ngrams asked
        for j=1, 50 do
            indexN = data[{i,j}]
            if gram_size == 1 then
                -- Format of indexes is always string
                indexes = tostring(indexN)
            end
            -- case word never seen in that context: smoothing only
            if F_c_w_table[gram_size][indexes][indexN] == nil then
                distribution[{i,j}] = alpha
            else
                distribution[{i,j}] = F_c_w_table[gram_size][indexes][indexN] + alpha
            end
        end

        -- case uniform prediction (because no word present in current context)
        if distribution:narrow(1,i,1):sum(2)[{1,1}] == 50 * alpha then
            -- TODO: try with previous context
            -- Select the first one (most common)
            distribution[{i,1}] = 1 + alpha
            --print('here', i, j, gram_size, indexes, indexN)
        end
    end
    -- normalization (ie we do the MLE given only the 50 possibilities)
    distribution:cdiv(torch.expand(distribution:sum(2), N_data, 50))
    
    return distribution
end

In [8]:
function perplexity(distribution, true_words)
    -- exp of the average of the cross entropy of the true word for each line
    -- true words (N_words to predict, one hot true value among 50)
    local perp = 0
    local N = true_words:size(1)
    for i = 1,N do
        mm,aa = true_words[i]:max(1)
        perp = perp + math.log(distribution[{i, aa[1]}])
    end
    perp = math.exp(- perp/N)
    return perp
end

In [32]:
-- Test of new mle version
distribution_mle_2_new = mle_proba_2(validation_2, 2, 1)

1-grams.hdf5	


2-grams.hdf5	


In [34]:
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2_new, validation_output))

Result on alpha smoothing 2grams	8.8674198826233	


In [88]:
F_c_w, F_c, N_c = build_context_count(train_2)
distribution_mle_2 = mle_proba(validation_2, F_c_w, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2, validation_output))

Result on alpha smoothing 2grams	8.8680603647573	


In [38]:
-- Test of new mle version
distribution_mle_3_new = mle_proba_2(validation_3, 3, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_3_new, validation_output))

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


Result on alpha smoothing 2grams	17.059968522337	


In [10]:
F_c_w_table = {}
for i=1,3 do
    train = get_train(i)
    F_c_w, F_c, N_c = build_context_count_2(train)
    F_c_w_table[i] = F_c_w
end

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


In [11]:
validation_2:narrow(1, 3, 1)

Columns 1 to 13
  500   305  8425  5919  4090  4981  8611  8244  4804  2009  9910   471     6

Columns 14 to 26
 9904  1198   514  1061  5049  6289  8585  1697  4648  3676  7371  1867  4857

Columns 27 to 39
 3924  6939  8431  3086  4288  3138  4486  3949  3730  8607  7310  9582  9873

Columns 40 to 51
 6426  7482  1215  6676  9535  1417  6268  7276  1861  4232  7448  3884
[torch.LongTensor of size 1x51]



In [92]:
F_c_w_table[1]['6594']

{
  6594 : 10
}


In [7]:
F_c_w, F_c, N_c = build_context_count(train_3)
distribution_mle_3 = mle_proba(validation_3, F_c_w, 1)

In [8]:
F_c_w, F_c, N_c = build_context_count(train_4)
distribution_mle_4 = mle_proba(validation_4, F_c_w, 1)

In [11]:
-- Results on the validation set
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2, validation_output))
print('Result on alpha smoothing 3grams', perplexity(distribution_mle_3, validation_output))
print('Result on alpha smoothing 4grams', perplexity(distribution_mle_4, validation_output))

Result on alpha smoothing 2grams	8.8680603647573	


Result on alpha smoothing 3grams	21.513973503713	


Result on alpha smoothing 4grams	29.942969799245	


# Witten Bell Model

In [ ]:
function compute_wb_line(N, entry, F_c_w_table, F_c_table, N_c_table, alpha)
    -- Return vector (50) predicting the distribution from entry
    -- N represent the Ngram size used in the prediction so context is N-1 gram
    local prediction = torch.zeros(50)
    local indexN
    -- case where computation only on the prior
    if N == 1 then
        for j=1,50 do
            indexN = entry[{1, j}]
            -- Corner case when prediction on words not on the dict (case for <s>)
            if F_c_w_table[1][tostring(indexN)] == nil then
                prediction[j] = 0
            else
                prediction[j] = F_c_w_table[1][tostring(indexN)][indexN]
            end
        end
        -- Normalizing
        return prediction:div(prediction:sum(1)[1])
    else
        -- Compute the MLE for current N
        -- context (at least with one element)
        local indexes = tostring(entry[{1, entry:size(2)}])
        for j=entry:size(2) - 1, entry:size(2) - 1 - (N-3), -1 do
            indexes = tostring(entry[{1, j}]) .. '-' .. indexes
        end
        -- check if context is unseen, otherwise go to next context
        if F_c_w_table[N][indexes] == nil then
            --print('unseen context')
            return compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha)
        end
        -- Compute MLE for each word
        for j=1,50 do
            indexN = entry[{1, j}]
            if F_c_w_table[N][indexes][indexN] == nil then
                prediction[j] = alpha
            else
                prediction[j] = F_c_w_table[N][indexes][indexN] + alpha
            end
        end
        
        -- Check that MLE predicted at least one words, otherwise go to next context
        if prediction:sum(1)[1] == 50 * alpha then
            --print('unseen words')
            return compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha)
        end
        
        -- Combining with next context
        prediction:add(compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha):mul(N_c_table[N][indexes]))
        -- Normalization
        -- TODO: We normalize as we apply on a reduced dataest (50 words)
        prediction:div(prediction:sum(1)[1])
        return prediction
    end
end

In [ ]:
-- Witten Bell: new version, computation done at once line by line
--
-- p_wb(w|c) = (F_c_w + N_c_. * p_wb(w|c'))/(N_c_. + F_c_.)
function distribution_proba_WB(N, data, alpha)
    local N_data = data:size(1)
    local M = data:size(2)

    -- Building the count matrix for each ngram size lower than N.
    local F_c_w_table = {}
    local F_c_table = {}
    local N_c_table = {}
    for i=1,N do
        train = get_train(i)
        F_c_w_table[i], F_c_table[i], N_c_table[i] = build_context_count_2(train)
    end

    -- Vector initialisation
    local distribution = torch.zeros(N_data, 50)
    for i=1,N_data do
        -- Compute witten bell for the whole line i
        distribution:narrow(1, i, 1):copy(compute_wb_line(N, data:narrow(1,i,1), F_c_w_table, F_c_table, N_c_table, alpha))
    end
    return distribution
end

In [269]:
t = compute_wb_line(2, validation_3:narrow(1,6,1), F_c_w_table, F_c_table, N_c_table, 1)

unseen words	


In [257]:
distribution_2 = distribution_proba_WB(2, validation_2, 1)

1-grams.hdf5	


In [ ]:
distribution_1 = distribution_proba_WB(1, validation_2, 1)
print('Result on alpha smoothing 1grams', perplexity(distribution_1, validation_output))
distribution_2 = distribution_proba_WB(2, validation_2, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_2, validation_output))
distribution_3 = distribution_proba_WB(3, validation_3, 1)
print('Result on alpha smoothing 3grams', perplexity(distribution_3, validation_output))
distribution_4 = distribution_proba_WB(4, validation_4, 1)
print('Result on alpha smoothing 4grams', perplexity(distribution_4, validation_output))

In [253]:
math.exp(1.94632)

7.0028695446154	


# Accuracy

In [17]:
m, true_output = validation_output:max(2)

In [21]:
function compute_accuracy(pred, true_output)
    max,argmax = pred:max(2)
    acc = 0
    for i = 1, true_output:size(1) do
        if argmax[i][1] == true_output[i][1] then
            acc = acc + 1
        end
    end
    score = acc/true_output:size(1)
    
    return score
end

In [101]:
print('Result on alpha smoothing ', compute_accuracy(distribution, true_output))
print('Result on alpha smoothing ', compute_accuracy(distribution_3, true_output))
print('Result on alpha smoothing ', compute_accuracy(distribution_4, true_output))
print('Result on Witten Bell ', compute_accuracy(d_wB, true_output))
print('Result on Witten Bell ', compute_accuracy(d_wB_no, true_output))


Result on alpha smoothing 	

0.59258160237389	


Result on alpha smoothing 	0.37952522255193	


Result on alpha smoothing 	0.23412462908012	


Result on Witten Bell 	0.32640949554896	


Result on Witten Bell 	0.25786350148368	


# Perplexity

In [125]:
print('Result on alpha smoothing ', perplexity(distribution))
print('Result on alpha smoothing ', perplexity(distribution_3))
print('Result on alpha smoothing ', perplexity(distribution_4))
print('Result on Witten Bell ', perplexity(d_wB))
print('Result on Witten Bell ', perplexity(d_wB_no))

Result on alpha smoothing 	110.46837660031	


Result on alpha smoothing 	64.899935042845	


Result on alpha smoothing 	59.51012776219	


Result on Witten Bell 	5.52172116585	


Result on Witten Bell 	inf	


In [15]:
train_1

  5053     12
  5884     10
  6722      9
   348    286
  7234      8
  5582     12
   245    397
  7827      7
  3784     21
  3442     24
  4830     15
  7235      7
  7236      8
  5885     11
  9933      3
  1487     69
  4831     12
  4062     18
  5886     11
  8571      2
  2470     37
  7237      8
  9585      5
   911    111
  7238      8
  5280     13
  3557     23
  7828      7
  9861      4
  7829      6
  2248     41
  7830      7
  4063     14
  5887     11
  7831      7
   336    297
  7239      7
  7240      8
  4832     15
  8572      6
  1264     83
  7832      7
  3921     19
  8573      6
  5888     11
   988    108
  7833      7
  2520     36
  7834      7
  6723      9
  4608     16
  4064     19
  4065     16
   349    246
  8574      6
   222    430
   639    154
  5889     11
  5890      9
  6724      9
  7241      7
  5281     12
  4221     18
  5282     13
  2521     36
  1324     78
   659    159
  3145     27
   757    141
  9862      4
  8575      6
  2289

    8
  1108     95
  3927     20
  6733      9
  4837     15
  7855      7
  8599      5
  4411     16
  6734      9
  6286     10
  1991     49
  2023     47
  6287     10
  5900     11
  3444     23
  7856      7
   716    148
  8600      6
  1927     50
  7258      8
  5057     14
  3244     26
  2331     36
  4617     16
  7857      7
  6288     10
    57   1731
  7259      8
  2294     41
   258    370
  7858      7
  1992     49
  8601      6
  3928     20
  5287     13
  3245     26
  5288     13
  4412     15
  5901     11
  3789     16
  7260      8
  2332     40
  4413     13
  7859      7
   930    115
   575    160
  9593      5
  8602      6
  7860      7
  4070     19
  3246     26
  1701     59
  7261      7
  2834     31
  4414     17
  1353     75
  2162     43
  8603      6
  5902     11
  3666     22
  5058     14
   412    256
  3341     25
  1235     85
  8604      6
   599    181
  5289     13
  7262      8
  3667     22
  6289     10
  1050     99
  7263      4


6749      4
  1673     60
  6750      7
  2082     46
  9600      5
  6299     10
  4843     14
     2  43343
  4073     18
  4074     19
  3148     27
  3562     22
  2473     37
  9601      5
  5593     12
  7878      7
  3149     27
  1411     72
  2587     35
  1392     73
  4075     19
  5594     12
  6300     10
  1015    105
  4624     16
  7277      8
  7278      8
   796    136
  4076     19
  7279      8
  2588     30
  2333     40
    11   9784
  1902     52
  5298      9
  1132     94
  2296     41
  3793     21
   828    131
  6751      8
  1412     73
  7280      8
   105    825
  1813     55
  6752      9
  3249     21
  1274     82
  9865      3
  7281      8
  7879      6
  8628      6
  5299     13
  7282      7
  6301      9
  6302     10
  7880      7
  8629      6
  4416     16
  8630      6
  2165     44
  7881      7
   876    108
  7283      8
    67   1288
  4226     17
  7284      7
  3931     20
  1506     68
  8631      6
  6303     10
  1236     85
  7285  

 43
  7296      8
   509    214
  8651      5
  7897      7
  7898      7
  2425     34
  6766      8
  7297      8
   306    324
  3154     20
  7899      7
   952    114
  2775     32
  7298      8
  7299      8
  5914     11
   119    753
  5915      9
  3348     25
  2837     30
  2589     31
  5067     14
  1464     70
  3797     21
  6767      9
  2531     25
  8652      6
   953    108
  7300      8
  7301      8
   211    439
  5068     14
  7302      7
  7900      7
  6313      2
  9610      5
  6314     10
  4234     18
  9611      5
  3253     26
  4424     14
  9612      5
  7901      7
  9613      5
  5304     12
  3066     28
  7902      7
  8653      6
  1958     50
  2532     33
  5916     11
  6768      9
  8654      6
   198    442
  7303      8
  1525     67
  5917      6
  1275     79
  7304      8
  7305      6
  4847     15
  4080     17
  7903      7
  3254     26
  1702     59
  6315      9
  1842     53
  1490     69
  4425     15
  2776     31
  1995     48
  

75      6
  4240     18
  4852     15
   523    202
  7322      8
  7921      7
  8676      6
  4634     15
  1843     54
  3068     28
  1844     53
  7922      4
  3941     18
   696    145
  1675     59
  8677      6
   427    248
  8678      4
  1636     60
  6783      9
  5611     12
    33   3334
    21   4480
    20   4546
  8679      6
  3351     25
  9617      5
  5612     11
  3069     28
  4241     18
  3258     23
  2590     34
   288    323
   944    113
  6325     10
  2654     32
  7923      7
  7924      3
  5073     14
  5314     13
  2655     34
  4084     15
  8680      6
  1526     67
  6784      9
  1782     49
   931    100
   990    110
  7925      7
  7926      7
  7927      7
  4242     18
  6326     10
  2905     30
  6785      9
  8681      6
   660    159
  3675     14
  8682      2
  6327     10
     7  23486
  8683      4
  3259     25
  7928      7
   309    334
  4429     17
   851    128
  3676     22
  2054     47
  4635     15
  9618      5
  8684    

...

2020     48
  2827     31
  1869     52
  9465      3
  6226     11
  3904     21
  4205     15
  5539     12
  3138     26
   940    111
  6227     11
  6228     11
  7754      8
   158    572
  8482      7
  1389     63
  7755      8
  1231     86
  8483      6
  1129     95
  2198     38
  9466      6
   292    341
  7172      9
  3774     22
  7756      6
  6229     11
  3139     28
  2326     39
  8484      7
  6656     10
  1156     77
  3327     26
  7757      8
  9919      4
  6230     11
  1408     73
  5022     15
  7758      8
  7759      1
  5252     14
  9467      6
  5023     15
  5540      9
  1522     68
  1809     44
   116    620
  3052     29
  1582     47
  5253     14
  4041     19
  2701     34
  5855     12
  9468      6
  8485      7
  1350     77
  5541     12
  6657     10
  7760      7
  4797     15
  7761      6
  6658     10
  3905     20
  7173      9
  8486      7
  1078     99
  9920      4
  5542     13
  9469      6
  3430     25
  7174      9
  3548  

  6
  1603     58
  1323     77
  1367     76
  6241     11
  7186      7
  7773      8
  4396     18
  9494      6
  1033    104
  1034     95
  6242     11
  7774      8
  3551     23
  1911     52
  2703     34
  7775      8
  7776      8
   670    124
  4208     19
  9829      5
  9926      4
  1523     65
  6673      8
   827    127
  5554     13
  1629     59
  6674     10
  4804     16
  9927      4
  9495      6
  9496      6
  9830      5
  8505      7
  1775     55
  5261     14
  8506      7
  3233     26
  5030     15
  9497      6
   474    222
    51   1741
   778    138
   555    194
  3910     19
  3656     15
  2328     31
  8507      7
  5262     13
   303    339
  9831      5
  6244     11
   808    129
  6245      8
  1566     66
  5263     14
  5859     12
  7777      8
  1460     69
  7187      8
  5264     14
  2762     33
  1650     60
  9498      6
   537    197
    28   3907
  7188      7
  9499      6
  2891     30
   414    256
  5265     14
  5555     13
  

67     13
  2893     31
  2579     35
  8520      6
  9528      6
   186    491
  6692      8
  4211     19
  7204      8
  2368     40
   357    295
  2117     46
  4052     20
  2580     34
  8521      7
  4598     17
   234    409
  3237     27
  7791      8
  3143     19
  6693     10
  8522      7
  5866     12
   501    183
   903    122
  2286     41
  3911     16
  4810     16
  9529      6
  4811     16
  2971     29
  3776     22
  2464     38
   252    388
  7792      8
  7793      7
  9530      6
  6256     11
  9840      4
  9531      6
  5036     15
   538    204
  8523      7
  5561     13
  5037     15
  9532      6
  4212     19
  8524      7
  6694      9
  7205      9
  4812     16
  6257     11
  9533      6
  6695     10
   378    279
  8525      7
  1307     80
  7794      8
  8526      7
   668    164
  2245     43
  9534      6
  1049    101
  4813     16
  7206      9
  6258     11
  7795      8
  5268     14
   769    141
  9535      5
  5038     12
  5562    

8
  3779     20
  9851      5
  9852      5
   304    338
  3916     21
    43   2353
   195    471
  9971      2
  4217     19
  7714      8
  4601     17
  8542      7
    58   1673
  6269     11
  7217      9
  7801      6
   764    103
  5276     14
  7802      8
  6710     10
  9560      6
  2829     32
  2708     25
  7803      8
  1604     64
  7218      9
  2765     33
  5569     13
  4819     16
  6711     10
   347    301
  9561      6
  1352     77
  1157     82
  1263     80
  9853      5
  3780     22
  9562      6
  1486     64
  8543      7
  6270     11
  5570     13
  3781     22
  1926     52
  9854      5
  1568     60
  7219      9
  3240     27
  9563      6
  2975     29
  8544      1
  1872     53
  5571     13
  1461     71
  9956      3
  1750     58
  5572     13
  5573     13
  7804      8
  6712     10
  2976     30
  4602     17
  7805      7
  5574     13
  6271     11
   809    135
   547    197
  5046     15
   748    144
  7220      7
  7806      8
  58

     45
  4826     16
  8567      7
  6720      9
  7232      9
  8568      7
  2369     40
  6277     11
  8569      7
   397    227
  5881     10
  3555     24
  4220     19
   161    574
  3556     24
   371    275
   441    243
   632    172
  9858      5
  1234     86
  7825      8
  8570      7
  5279     14
  2981     26
  2049     48
  1370     70
  5052     15
   929    118
  5882      8
  9859      5
  4827     16
  2647     32
    63   1467
  2583     36
  2203     44
  4828     16
  4829     16
  3920     21
   626    173
  2329     41
  9584      6
  5883     11
  7233      9
  2519     36
  4407     18
  4408     18
  9860      5
  7826      8
  6278     11
  6721      1
[torch.LongTensor of size 9997x2]

